<a href="https://colab.research.google.com/github/marsggbo/AutoMLDemos/blob/master/ch4/nasbench101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 下载安装

In [ ]:
!git clone https://github.com/marsggbo/hyperbox.git

In [ ]:
cd hyperbox

In [ ]:
!pip install -e .

# 2. 下载查询数据库

In [ ]:
%%bash
pip install peewee
pip install gdown

if [ -z "${NASBENCHMARK_DIR}" ]; then
    NASBENCHMARK_DIR=~/.hyperbox/nasbenchmark
fi

echo "Downloading NAS-Bench-201..."
if [ -f "nb201.pth" ]; then
    echo "nb201.pth found. Skip download."
else
    gdown https://drive.google.com/uc\?id\=16Y0UwGisiouVRxW-W5hEtbxmcHw_0hF_ -O nb201.pth
fi

echo "Generating database..."
rm -f ${NASBENCHMARK_DIR}/nasbench201.db ${NASBENCHMARK_DIR}/nasbench201.db-journal
mkdir -p ${NASBENCHMARK_DIR}

接下来我们运行如下代码解析NAS-Bench-201数据集（`nb201.pth`文件）。

注意这个文件比较大，所以对内存要求比较大，实测Google Colab免费版本无法正常运行，建议在CPU内存更大的设备上运行。

In [ ]:
!python hyperbox/networks/nasbench201/db_gen/db_gen.py nb201.pth

# 3. 使用NAS-Bench-201数据集

In [2]:
# 1. 引入相关包

import torch

from hyperbox.mutator import RandomMutator
from hyperbox.networks.nasbench201 import NASBench201Network
from hyperbox.networks.nasbench201.db_gen import query_nb201_trial_stats
device = 'cuda' if torch.cuda.is_available() else 'cpu'


/Users/marsggbo/opt/anaconda3/envs/hyperbox/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 2. 构建搜索空间和随机搜索策略

net = NASBench201Network().to(device)
rm = RandomMutator(net)


In [4]:
# 3. 随机采样并输出模型结构
for i in range(3):
    rm.reset()
    print(0, net.print_cell_mask_by_idx(0))
    print(2, net.print_cell_mask_by_idx(2))
    print(4, net.print_cell_mask_by_idx(4))
    a = torch.rand(2, 3, 64, 64).to(device)
    b = net(a)
    print(net.arch_size((2,3,64,64)))

0_1 tensor([False, False,  True, False, False])
0_2 tensor([False, False,  True, False, False])
1_2 tensor([False,  True, False, False, False])
0_3 tensor([False, False, False, False,  True])
1_3 tensor([False,  True, False, False, False])
2_3 tensor([ True, False, False, False, False])
0 None
0_1 tensor([False, False,  True, False, False])
0_2 tensor([False, False,  True, False, False])
1_2 tensor([False,  True, False, False, False])
0_3 tensor([False, False, False, False,  True])
1_3 tensor([False,  True, False, False, False])
2_3 tensor([ True, False, False, False, False])
2 None
0_1 tensor([False, False,  True, False, False])
0_2 tensor([False, False,  True, False, False])
1_2 tensor([False,  True, False, False, False])
0_3 tensor([False, False, False, False,  True])
1_3 tensor([False,  True, False, False, False])
2_3 tensor([ True, False, False, False, False])
4 None
(314.24544, 2.1338882446289062)
0_1 tensor([False, False, False,  True, False])
0_2 tensor([False,  True, False, Fa

In [ ]:
# 4. 随机采样并输出模型的信息
for i in range(3):
    rm.reset()
    arch_json = net.arch 
    acc = net.query_by_key()
    print(acc)
    # 输出在cifar10数据集上训练200个epoch后的结果
    for t in query_nb201_trial_stats(arch_json, 200, 'cifar10'):
        print(t)